In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import zipfile
from keras.layers import Dense, Reshape
import matplotlib.pyplot as plt
import math
from keras.losses import *
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape
from keras.layers import LSTM, Flatten, Input, Embedding, concatenate, Concatenate, average, maximum, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
def balanceFraudDataframe(df, p): #p is desired fraud/non-fraud ratio
    dfFraud = df.loc[df['isFraud']==1,:]
    x = len(dfFraud)
    n = len(df)
    numOfTimesToResample = int((p * n - x)/((1-p) * x))
    #print(numOfTimesToResample)
    dfBalanced = df
    for _ in range(numOfTimesToResample):
        dfBalanced = dfBalanced.append(dfFraud)   
    return dfBalanced

In [3]:
def findPercentFraud(df):
    return len(df.loc[df.isFraud == 0, :])/len(df)

In [4]:
df = pd.read_pickle("data/dataframe_train_v09.pd")
df_fraud = pd.read_pickle('dfFraud.pd')

In [5]:
df['isFraud'] = df_fraud['isFraud']

In [6]:
columnsNotToScale = list(df.columns[(df.dtypes == 'bool') | (df.dtypes=='uint8')])
#columnsNotToScale.append('isFraud')

In [7]:
df.drop(columnsNotToScale, axis=1).shape

(590540, 402)

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaler.fit(df.drop('isFraud', axis=1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [9]:
np.random.seed(1234)
choose = np.random.rand(len(df))
dfTrain = df.loc[choose >= .2, :]
dfTest = df.loc[choose < .2, :]

In [10]:
dfTrain.shape

(472515, 750)

In [11]:
assert len(dfTrain) + len(dfTest) == len(df)

In [12]:
print(findPercentFraud(dfTrain))
dfTrain = balanceFraudDataframe(dfTrain, 0)
print(findPercentFraud(dfTrain))

0.964932330190576
0.964932330190576


In [13]:
print(findPercentFraud(dfTest))

0.9653209065875874


In [14]:
x_train = np.array(scaler.transform(dfTrain.drop('isFraud', axis=1)))
y_train= np.array(dfTrain['isFraud'])
#x_test = scaler.transform(dfTest.drop(["isFraud"], axis=1))
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler2.fit(dfTest.drop('isFraud', axis=1))

x_test = np.array(scaler2.transform(dfTest.drop('isFraud', axis=1)))
y_test = np.array(dfTest['isFraud'])

In [ ]:
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')

In [ ]:
from sklearn.utils import class_weight
from sklearn.metrics import precision_recall_fscore_support
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(y_true))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(y_pred))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


inputs = Input(shape=(749,))
# We stack a deep densely-connected network on top
x = Dense(600, activation='relu', use_bias=True)(inputs)
x = Dense(600,  activation='relu', use_bias=True)(x)
#x = Dropout(.2)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
#x = Dropout(.1)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
#x = Dropout(.2)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
x = Dense(200, activation='relu', use_bias=True)(x)
# And finally we add the main logistic regression layer
output = Dense(1 , activation= "sigmoid", name='ANN_output')(x)

adam = optimizers.Adam(lr=0.001)
model = Model(inputs=inputs, outputs = output)

def mean_pred(y_true, y_pred):
    return tf.nn.weighted_cross_entropy_with_logits( y_true, logits=y_pred, pos_weight=.0050)


# def metrics_cust(y_true, y_pred):
#     return tf.metrics.recall(y_true, y_pred)

#weights = class_weight.compute_class_weight('balanced',
#                                            np.unique(y_train),
#                                            y_train)
# weights = [4, 1]
from sklearn.metrics import roc_auc_score

model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy', recall_m, precision_m, f1_m])
model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2, )#class_weight= weights)


Epoch 1/50
 - 21s - loss: 0.1201 - acc: 0.9673 - recall_m: 0.1449 - precision_m: 0.6542 - f1_m: 0.2260
Epoch 2/50
 - 19s - loss: 0.1073 - acc: 0.9712 - recall_m: 0.2392 - precision_m: 0.8195 - f1_m: 0.3624
Epoch 3/50
 - 19s - loss: 0.1042 - acc: 0.9722 - recall_m: 0.2707 - precision_m: 0.8222 - f1_m: 0.3992
Epoch 4/50
 - 19s - loss: 0.1023 - acc: 0.9727 - recall_m: 0.2860 - precision_m: 0.8229 - f1_m: 0.4158
Epoch 5/50
 - 19s - loss: 0.1002 - acc: 0.9731 - recall_m: 0.2989 - precision_m: 0.8261 - f1_m: 0.4302
Epoch 6/50
 - 19s - loss: 0.0988 - acc: 0.9736 - recall_m: 0.3144 - precision_m: 0.8299 - f1_m: 0.4480
Epoch 7/50
 - 19s - loss: 0.0977 - acc: 0.9740 - recall_m: 0.3230 - precision_m: 0.8479 - f1_m: 0.4592
Epoch 8/50
 - 19s - loss: 0.0967 - acc: 0.9741 - recall_m: 0.3280 - precision_m: 0.8510 - f1_m: 0.4650
Epoch 9/50
 - 19s - loss: 0.0952 - acc: 0.9745 - recall_m: 0.3408 - precision_m: 0.8420 - f1_m: 0.4780
Epoch 10/50
 - 19s - loss: 0.0946 - acc: 0.9746 - recall_m: 0.3419 - prec

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=500, verbose=2, )

Epoch 1/20
 - 21s - loss: 0.0674 - acc: 0.9814 - recall_m: 0.5384 - precision_m: 0.8881 - f1_m: 0.6604
Epoch 2/20
 - 20s - loss: 0.0653 - acc: 0.9820 - recall_m: 0.5505 - precision_m: 0.8985 - f1_m: 0.6727
Epoch 3/20
 - 20s - loss: 0.0639 - acc: 0.9824 - recall_m: 0.5627 - precision_m: 0.9001 - f1_m: 0.6833
Epoch 4/20
 - 20s - loss: 0.0627 - acc: 0.9827 - recall_m: 0.5697 - precision_m: 0.9036 - f1_m: 0.6899
Epoch 5/20
 - 21s - loss: 0.0619 - acc: 0.9829 - recall_m: 0.5815 - precision_m: 0.8970 - f1_m: 0.6963
Epoch 6/20
 - 20s - loss: 0.0609 - acc: 0.9831 - recall_m: 0.5823 - precision_m: 0.9042 - f1_m: 0.6995
Epoch 7/20
 - 20s - loss: 0.0606 - acc: 0.9832 - recall_m: 0.5846 - precision_m: 0.9045 - f1_m: 0.7020
Epoch 8/20
 - 20s - loss: 0.0595 - acc: 0.9835 - recall_m: 0.5955 - precision_m: 0.9029 - f1_m: 0.7078
Epoch 9/20
 - 20s - loss: 0.0601 - acc: 0.9835 - recall_m: 0.5952 - precision_m: 0.9064 - f1_m: 0.7095
Epoch 10/20
 - 20s - loss: 0.0579 - acc: 0.9839 - recall_m: 0.6056 - prec

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=5000, verbose=2, )

Epoch 1/20
 - 19s - loss: 0.0409 - acc: 0.9888 - recall_m: 0.7152 - precision_m: 0.9542 - f1_m: 0.8169
Epoch 2/20
 - 19s - loss: 0.0367 - acc: 0.9901 - recall_m: 0.7504 - precision_m: 0.9578 - f1_m: 0.8411
Epoch 3/20
 - 19s - loss: 0.0353 - acc: 0.9905 - recall_m: 0.7640 - precision_m: 0.9577 - f1_m: 0.8495
Epoch 4/20
 - 19s - loss: 0.0340 - acc: 0.9909 - recall_m: 0.7730 - precision_m: 0.9608 - f1_m: 0.8564
Epoch 5/20
 - 19s - loss: 0.0332 - acc: 0.9912 - recall_m: 0.7794 - precision_m: 0.9615 - f1_m: 0.8605
Epoch 6/20
 - 19s - loss: 0.0327 - acc: 0.9914 - recall_m: 0.7830 - precision_m: 0.9645 - f1_m: 0.8638
Epoch 7/20


In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, model.predict(x_test))

In [ ]:
model.save("model_sub2.h5")

In [ ]:
df_subm = pd.read_pickle('./data/dataframe_test_v09.pd')
x_sub = scaler.transform(df_subm)
y_pred = model.predict(x_sub)
dfToSubmit = pd.DataFrame(y_pred)
dfToSubmit.columns = ['isFraud']
dfToSubmit.index = df_subm.index
dfToSubmit

In [ ]:
dfToSubmit.to_csv('./final_submission.csv')

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#trainScore = numpy.mean(numpy.abs((y_train - trainPredict))/y_train)

print('Train Score: %.2f MAPE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test, testPredict))
#testScore =  numpy.mean(numpy.abs((y_test - testPredict))/y_test)

print('Test Score: %.2f MAPE' % (testScore))




In [ ]:
from imblearn.under_sampling import TomekLinks
sampler = TomekLinks()

In [ ]:
n = 10
l = df.shape[0]//n
X_ress = []
y_ress = []
for i in range(n):
    print(i)
    X_res, y_res = sampler.fit_resample(df[i*l:(i+1)*l], df_fraud[i*l:(i+1)*l].values.reshape(-1))
    X_ress.append(X_res)
    y_ress.append(y_res)

In [ ]:
X_res = np.vstack(X_ress)
y_res = np.hstack(y_ress)

In [ ]:
X_res.shape

In [ ]:
X_res, y_res = sampler.fit_resample(df, df_fraud.values.reshape(-1))

In [ ]:
X_res.shape[0]/df.shape[0]

In [ ]:
import pickle
with open('./data/dataframe_train_v07.np', 'wb') as f:
    pickle.dump(X_res, f)

In [ ]:
with open('./dfFraud.np', 'wb') as f:
    pickle.dump(y_res, f)

In [ ]:
df_fraud_res = pd.DataFrame(y_res, X_res[:,0].astype('int32'), df_fraud.columns)#, [str(i) for i in df_fraud.dtypes])

In [ ]:
df_res = pd.DataFrame?
#(X_res, X_res[:,0].astype('int32'), df.columns, dtype=

In [ ]:
df_res = pd.DataFrame

In [ ]:
for column in df.columns:
    df_res[column] = df_res[column].astype(df[column].dtypes)

In [ ]:
df_res.to_pickle('data/dataframe_train_v07.pd')

In [ ]:
df_fraud_res.to_pickle('./dfFraud_v07.pd')

In [ ]:
df.dtype